In [62]:
import csv
import datetime
import requests
import time
from bs4 import BeautifulSoup 

In [63]:
def call_api(topic,page=0):
    
    queryParams = {}
    queryParamsLabel = ('q','api-key','fq','page')
    # q represnts the topic or the type of news that the search query should return results for
    # api-key represents the user key
    # fq represents the source from where news we want to fetch
    # page is used for pagination
    
    queryParams[queryParamsLabel[0]] = topic
    
    queryParams[queryParamsLabel[1]] = 'nytimes api key'
    
    queryParams[queryParamsLabel[2]] = 'The New York Times'
    queryParams[queryParamsLabel[3]] = page
    
    #print(queryParams)
    page_response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json" , params = queryParams)
    
    return page_response

def write_articles_to_csv(articles_to_write,filename):
    if len(articles_to_write) != 0:
        headers = articles_to_write[0].keys()
        with open(filename, "w") as output_file:
            dict_writer = csv.DictWriter(output_file, articles_to_write[0].keys())
            dict_writer.writeheader()
            dict_writer.writerows(articles_to_write)

def fetch_real_news(topics):
 
    news_details = []
    headlines = []
    
    for topic in topics:
        # print("topic",topic)
        max_page_count = 100
        page_counter = 0
        while max_page_count > 0:
            
            page_response = call_api(topic,(page_counter))
            max_page_count = max_page_count - 1
            page_counter = page_counter +1
            if page_response.status_code==200:
                # print("response obtained")
                response = page_response.json()      
                articles = response['response']['docs']
                
                for article in articles:                
                    article_details = {}
                    article_headlines_details = {}

                    date_time_str =  article['pub_date'].split("T")[0] 
                    parse_string ="%Y-%m-%d"
                    date_time_obj = datetime.datetime.strptime(date_time_str, parse_string)

                    web_url_article = article["web_url"]
                    article_text = call_api_article(web_url_article)
                    
                    if article_text == None or article_text =='':
                        article_details['statement'] = article['lead_paragraph']
                    else:
                        article_details['statement'] = article_text
                    
                    article_details['date'] = date_time_obj.date()
                    article_details['label'] = "real"

                    article_headlines_details['date'] = date_time_obj.date()
                    article_headlines_details['label'] = "real"
                    article_headlines_details['statement'] = article['headline']['main']

                    news_details.append(article_details)
                    headlines.append(article_headlines_details)
            else:
                break
            time.sleep(10)

    print("done",len(news_details))
    write_articles_to_csv(news_details,"real_news.csv")
    write_articles_to_csv(headlines,"real_news_headlines.csv")
    
    
def call_api_article(web_url_article):
    
    page_response = requests.get(web_url_article)
    
    articles_details = ""
    
    if page_response.status_code==200:
        
        article_text = BeautifulSoup(page_response.content, 'html.parser')
     
        article_paragraphs = article_text.select("div.css-53u6y8")
        
        for article_paragraph in article_paragraphs:
            p_tag_text = article_paragraph.select("p.css-1ygdjhk.evys1bk0")
            
            for p_tag in p_tag_text:
                if p_tag.text ==None or p_tag.text == '' or p_tag.text == "_____________________":
                    continue
                else:
                    articles_details = articles_details + p_tag.text
            
            paragraphs = article_paragraph.select("em.css-2fg4z9.e1gzwzxm0")
            for paragraph in paragraphs:
                if paragraph.text == "_____________________":
                    continue
                articles_details = articles_details + paragraph.text
  
    return articles_details
    
    
    
    
    
    

In [64]:
fetch_real_news(["Politics"])

done 1000
